**Agentic AI: Homework Triage System with Input Guardrails**

1.   The Framework: OpenAI Agent Triage System.
2.   The Use Case: AI Homework Helper.
3.   The Safety Layer: With Guardrails.



**The primary purpose of this script is to create a Triage System that automatically routes user questions to specific experts (Math or History) while using a Guardrail to ensure only "homework-related" questions are processed.**


**Kiranmai Gunta
02/22/2026**

**Project Name: AI Homework Triage & Safety System**

**Skills: Python, OpenAI API, Agentic Workflows, Prompt Engineering**

**Description: >  Architected a multi-agent system using the OpenAI Agents SDK**

**Implemented Input Guardrails to ensure the system remains on-task (Homework focus)**

**Designed a Triage Agent to autonomously route tasks to subject-matter experts**

In [ ]:
%%bash
mkdir -p my_project
cd my_project
rm -rf .venv
python -m venv --without-pip .venv
curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
./.venv/bin/python get-pip.py
rm get-pip.py
./.venv/bin/python -m pip install --upgrade pip

  Using cached pip-26.0.1-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-26.0.1-py3-none-any.whl (1.8 MB)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2142k  100 2142k    0     0  6089k      0 --:--:-- --:--:-- --:--:-- 6102k


In [ ]:
%%bash
cd my_project
./.venv/bin/pip install openai-agents

  Using cached openai_agents-0.9.3-py3-none-any.whl.metadata (13 kB)
  Using cached griffe-1.15.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached mcp-1.26.0-py3-none-any.whl.metadata (89 kB)
  Using cached openai-2.21.0-py3-none-any.whl.metadata (29 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached types_requests-2.32.4.20260107-py3-none-any.whl.metadata (2.0 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached anyio-4.12.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jsonschema-4.26.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached pydantic_settings-2.13.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached pyjwt-2.11.0-py3-none-any.whl.metadata 

To run commands using the virtual environment's Python interpreter, you need to explicitly call the interpreter from its path within the virtual environment. Here's how you can check its Python version and list packages installed inside it:

In [ ]:
%%bash
cd my_project
./.venv/bin/python --version
./.venv/bin/pip list

Python 3.12.12
Package                   Version
------------------------- ---------------
annotated-types           0.7.0
anyio                     4.12.1
attrs                     25.4.0
certifi                   2026.1.4
cffi                      2.0.0
charset-normalizer        3.4.4
click                     8.3.1
colorama                  0.4.6
cryptography              46.0.5
distro                    1.9.0
griffe                    1.15.0
h11                       0.16.0
httpcore                  1.0.9
httpx                     0.28.1
httpx-sse                 0.4.3
idna                      3.11
jiter                     0.13.0
jsonschema                4.26.0
jsonschema-specifications 2025.9.1
mcp                       1.26.0
openai                    2.21.0
openai-agents             0.9.3
pip                       26.0.1
pycparser                 3.0
pydantic                  2.12.5
pydantic_core             2.41.5
pydantic-settings         2.13.1
PyJWT                     2.

In [ ]:
pip install openai-agents # or `uv add openai-agents`, etc

In [ ]:
%%bash
export OPENAI_API_KEY='YOUR_API_KEY_HERE'

In [ ]:
from agents import Agent

agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

In [ ]:
from agents import Agent

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

In [ ]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent]
)

In [ ]:
from agents import Runner

async def main():
    result = await Runner.run(triage_agent, "who was the first president of the united states?")
    print(result.final_output)

In [ ]:
from agents import GuardrailFunctionOutput, Agent, Runner
from pydantic import BaseModel


class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

- HomeworkOutput: A Pydantic model that forces the AI to provide a structured "Yes/No" (Boolean) answer and a reason.

- homework_guardrail: This is the "Security Gate." It uses the guardrail_agent to inspect the user's prompt. If the prompt doesn't look like homework, it sets tripwire_triggered=True, which halts the execution immediately.

- triage_agent: This is the "Dispatcher." If the guardrail passes, this agent looks at the question and hands it off to either the Math Tutor or the History Tutor.

In [ ]:
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from agents.exceptions import InputGuardrailTripwireTriggered
from pydantic import BaseModel
import asyncio
import nest_asyncio # Import nest_asyncio
import os # Import os to set environment variable

# Set the OpenAI API key explicitly for the Python environment
os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY_HERE'
# --- 1. DATA STRUCTURE DEFINITION ---
# Defines the expected structure for the guardrail's decision.
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str
# --- 2. AGENT DEFINITIONS ---
# This agent acts as the 'judge' to determine if an input is homework.
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)
# Specialist agents that provide the actual answers.
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)
philosophy_tutor_agent = Agent(
    name="philosophy Tutor",
    handoff_description="Specialist agent for philosiphycal questions",
    instructions="You provide assistance with philosophy queries. Explain important events and context clearly.",
)

# --- 3. GUARDRAIL LOGIC ---
# This function runs BEFORE the main agent handles the request.
async def homework_guardrail(ctx, agent, input_data):

# Runs the guardrail agent to classify the input
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)

# TRIPWIRE: If 'is_homework' is False, trigger the tripwire to block the input.
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

# --- 4. MAIN TRIAGE AGENT ---
# The entry point that decides which specialist to call, protected by the guardrail.
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent,philosophy_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

async def main():
    # Example 1: History question
    try:
        result = await Runner.run(triage_agent, "who was the first president of the united states?")
        print("History Question Result:", result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)
    # Example 2: General/philosophical question
    try:
        result = await Runner.run(triage_agent, "Define Phylosophy?")
        print("Philosophical Question Result:", result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)
    # Example 3: Mathematics
    try:
        result = await Runner.run(triage_agent, "solve x+y=8 x+z=9 what is x+y+z")
        print("Mathematics Question Result:", result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

if __name__ == "__main__":
    # Apply nest_asyncio to allow asyncio.run() in an already running event loop (e.g., Colab)
    # If nest_asyncio is not installed, run: !pip install nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())

History Question Result: The first president of the United States was George Washington. He served from 1789 to 1797. Washington was unanimously elected as the nation's first president and is often called the "Father of His Country." Before becoming president, he was the commander-in-chief of the Continental Army during the American Revolutionary War, playing a crucial role in the colonies’ victory over Britain. As president, he helped set many precedents for the office and oversaw the creation of key government institutions.
Philosophical Question Result: **Philosophy** is the study of fundamental questions about existence, knowledge, values, reason, mind, and language. It involves using critical thinking, logic, and reason to explore basic concepts such as reality, truth, morality, and the nature of being. Philosophers seek to understand and analyze ideas that form the foundation of human experience and society. 

In summary, philosophy asks and investigates the "big questions" about